#  微信医聊自动问答 WeChaty + PaddleHub
## 一、项目背景

基于wechaty和PaddleHub实现一个简单的中文医学问答功能。

wechaty使用相关参考项目: https://aistudio.baidu.com/aistudio/projectdetail/1847009

文本匹配参考项目: https://aistudio.baidu.com/aistudio/projectdetail/1456984

<iframe style="width:98%;height: 450px;" src="//player.bilibili.com/player.html?aid=460438217&bvid=BV1G541137zm&cid=334873181&page=1" scrolling="no" border="0" frameborder="no" framespacing="0" allowfullscreen="true"> </iframe>

**b站视频链接：[https://www.bilibili.com/video/BV1G541137zm/](https://www.bilibili.com/video/BV1G541137zm/)**

## 二、文本生成模型简介
* 模型概述

ERNIE-GEN 是面向生成任务的预训练-微调框架，首次在预训练阶段加入span-by-span 生成任务，让模型每次能够生成一个语义完整的片段。在预训练和微调中通过填充式生成机制和噪声感知机制来缓解曝光偏差问题。此外, ERNIE-GEN 采样多片段-多粒度目标文本采样策略, 增强源文本和目标文本的关联性，加强了编码器和解码器的交互。ernie_gen module是一个具备微调功能的module，可以快速完成特定场景module的制作。
![](https://ai-studio-static-online.cdn.bcebos.com/b80fe3dd29e54f948cb8127857d7b213a608ab1b894146d19ca7013bdcab5183)

* 模型链接

[https://www.paddlepaddle.org.cn/hubdetail?name=ernie_gen&en_category=TextGeneration](https://www.paddlepaddle.org.cn/hubdetail?name=ernie_gen&en_category=TextGeneration)

## 三、准备数据集

#### 1. 解压数据集

!unzip /home/aistudio/data//data52093/data4807.zip


####  2. 背景描述

中文医药方面的问答数据集，超过10万条。

 
####  3. 数据说明

questions.csv：所有的问题及其内容。

answers.csv ：所有问题的答案。

train_candidates.txt， dev_candidates.txt， test_candidates.txt ：将上述两个文件进行了拆分。

 
####  4. 数据来源

https://github.com/zhangsheng93/cMedQA2


####  5. 制作数据集

原数据集我已使用脚本处理过, 也可以自己使用脚本调整train和dev的比例， 相关工具脚本可使用data_prepare.py和cut_txt.py

在上方大家也看到了数据集格式为：

* train_path(str): 训练集路径。训练集的格式应为："序号\t输入文本\t标签"，例如："1\t床前明月光\t疑是地上霜"
* dev_path(str): 验证集路径。验证集的格式应为："序号\t输入文本\t标签"，例如："1\t举头望明月\t低头思故乡"

![](https://ai-studio-static-online.cdn.bcebos.com/f625d35b75f4437b8294002e1aba8cf728655f66e52543e2a789ae414a833b49)

        
        



In [5]:
#整理数据格式
#!python data_prepare.py 

#调整数据比例
#!cut_txt.py

#解压已处理好的数据
#!unzip data/data84360/list.zip -d ./

Archive:  data/data84360/list.zip
replace ./test_list.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


## 四、安装Fine-Tune环境

In [ ]:
!pip install paddle-ernie==0.0.4.dev1
!pip install  paddlehub==1.8.0 -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple/
  Created wheel for paddle-ernie: filename=paddle_ernie-0.0.4.dev1-cp37-none-any.whl size=21722 sha256=809bedd5bad94db7b8f73b662252e3dc57448e227addfcf1be2bd16713da79fb
  Stored in directory: /home/aistudio/.cache/pip/wheels/8a/b6/b9/58e36cdf58f588998b54d6f0233a176e537576bba8bbe10def
Successfully built paddle-ernie
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 337kB 11.5MB/s eta 0:00:01
  Found existing installation: paddlehub 2.0.4
    Uninstalling paddlehub-2.0.4:
      Successfully uninstalled paddlehub-2.0.4


Archive:  data/data84360/list.zip
  inflating: ./test_list.txt         
  inflating: ./train_list.txt        


## 五、模型微调Fine-Tune

微调API，

参数

* train_path(str): 训练集路径。训练集的格式应为："序号\t输入文本\t标签"，例如："1\t床前明月光\t疑是地上霜"
* dev_path(str): 验证集路径。验证集的格式应为："序号\t输入文本\t标签"，例如："1\t举头望明月\t低头思故乡"
* save_dir(str): 模型保存以及验证集预测输出路径。
* init_ckpt_path(str): 模型初始化加载路径，可实现增量训练。
* use_gpu(bool): 是否使用GPU。
* max_steps(int): 最大训练步数。
* batch_size(int): 训练时的batch大小。
* max_encode_len(int): 最长编码长度。
* max_decode_len(int): 最长解码长度。
* learning_rate(float): 学习率大小。
* warmup_proportion(float): 学习率warmup比例。
* weight_decay(float): 权值衰减大小。
* noise_prob(float): 噪声概率，详见ernie gen论文。
* label_smooth(float): 标签平滑权重。
* beam_width(int): 验证集预测时的beam大小。
* length_penalty(float): 验证集预测时的长度惩罚权重。
* log_interval(int): 训练时的日志打印间隔步数。
* save_interval(int): 训练时的模型保存间隔部署。验证集将在模型保存完毕后进行预测。

返回

* result(dict): 运行结果。包含2个键:

      last_save_path(str): 训练结束时的模型保存路径。
      last_ppl(float): 训练结束时的模型困惑度。



In [ ]:
import paddlehub as hub

module = hub.Module(name="ernie_gen")

result = module.finetune(
    train_path='/home/aistudio/train_list_new.txt',
    # dev_path='/home/aistudio/test_list_tiny.txt',
    save_dir="ernie_gen_ch_med",
    max_steps=40000,
    max_encode_len=256,
    max_decode_len=256,
    noise_prob=0.1,
    batch_size=24,
    log_interval=50,
    save_interval=2000
)

[2021-04-27 00:27:33,228] [    INFO] - Installing ernie_gen module
[2021-04-27 00:27:33,243] [    INFO] - Module ernie_gen already installed in /home/aistudio/.paddlehub/modules/ernie_gen
2021-04-27 00:27:33,246-INFO: get pretrain dir from https://ernie-github.cdn.bcebos.com/model-ernie1.0.1.tar.gz
[INFO 2021-04-27 00:27:33,246 tokenizing_ernie.py:89] get pretrain dir from https://ernie-github.cdn.bcebos.com/model-ernie1.0.1.tar.gz
[INFO] 2021-04-27 00:27:33,262 [feature_column.py:  349]:	reading raw files from /home/aistudio/train_list_new.txt
2021-04-27 00:27:33,264-INFO: get pretrain dir from https://ernie-github.cdn.bcebos.com/model-ernie1.0.1.tar.gz
[INFO 2021-04-27 00:27:33,264 modeling_ernie.py:196] get pretrain dir from https://ernie-github.cdn.bcebos.com/model-ernie1.0.1.tar.gz
2021-04-27 00:27:37,774-INFO: loading pretrained model from /tmp/466eabcffd6d6a83ae9cb97dd1a167bd
[INFO 2021-04-27 00:27:37,774 modeling_ernie.py:214] loading pretrained model from /tmp/466eabcffd6d6a83

## 六、导出module

参数

* params_path(str): 模型参数路径。
* module_name(str): module名称，例如"ernie_gen_couplet"。
* author(str): 作者名称。
* version(str): 版本号。
* summary(str): module的英文简介。
* author_email(str): 作者的邮箱地址。
* export_path(str): module的导出路径。

In [ ]:

module.export(params_path='ernie_gen_ch_med/step_40001_ppl_9.14520.pdparams', module_name="ernie_gen_ch_med_save", author="thunder95")

[2021-04-27 13:03:32,032] [    INFO] - Begin export the model save in ernie_gen_ch_med/step_40001_ppl_9.14520.pdparams ...
[2021-04-27 13:03:32,702] [    INFO] - The module has exported to /home/aistudio/ernie_gen_ch_med_save


## 七、安装新场景的module

In [ ]:
!hub install ernie_gen_ch_med_save

2021-04-27 13:03:55,042-INFO: font search path ['/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf', '/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/afm', '/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/pdfcorefonts']
[INFO 2021-04-27 13:03:55,042 font_manager.py:1071] font search path ['/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf', '/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/afm', '/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/pdfcorefonts']
2021-04-27 13:03:55,388-INFO: generated new fontManager
[INFO 2021-04-27 13:03:55,388 font_manager.py:1458] generated new fontManager


## 八、使用命令行预测
上方我们就已经安装成功了，这里我们进行命令行测试

In [ ]:
!hub run ernie_gen_ch_med_save --input_text="手和脚出汗，晚上失眠多梦是不是肾虚" --use_gpu True --beam_width 5

[['考虑是肾阴虚引起的症状，建议服用六味地黄丸和谷维素片以及黄芪精口服液治疗。另外多喝白开水，注意休息。', '考虑是肾阴虚引起的，建议服用六味地黄丸和谷维素片以及黄芪精口服液，另外多喝白开水，注意休息。', '考虑是肾阴虚引起的，建议服用六味地黄丸和谷维素片以及黄芪精口服液治疗。另外多喝白开水，注意休息。', '考虑是肾阴虚引起的症状，建议服用六味地黄丸和谷维素片以及黄芪精口服液，另外多喝白开水，注意休息。', '考虑是肾阴虚引起的症状，建议服用六味地黄丸和谷维素片以及黄芪精口服液治疗。另外多喝白开水，注意休息，']]


## 九、API预测
这里我们进行API测试，大家可以在下方查看结果

In [ ]:
import paddlehub as hub
import os

os.environ["CUDA_VISIBLE_DEVICES"] ="0"

module = hub.Module(name="ernie_gen_ch_med_save")

test_texts = ["我身上起小红点还痒猫耳疮， 怎么办"]
# generate包含3个参数，texts为输入文本列表，use_gpu指定是否使用gpu，beam_width指定beam search宽度。
results = module.generate(texts=test_texts, use_gpu=True, beam_width=1)
for result in results:
    print(result)

[2021-04-27 13:09:40,758] [    INFO] - Installing ernie_gen_ch_med_save module
[2021-04-27 13:09:40,788] [    INFO] - Module ernie_gen_ch_med_save already installed in /home/aistudio/.paddlehub/modules/ernie_gen_ch_med_save


['这个情况考虑是有毛囊炎的可能，建议你可以用高锰酸钾溶液清洗后涂抹百多邦软膏治疗，注意饮食，多喝水，多']


# 十、部署WeChaty服务端
这里使用docker脚本可方便快速部署。请将your_token处替换成你的WeChaty token (**puppet_padlocal_xxxxxxxxxxxxx)**

```
export WECHATY_LOG="verbose"
export WECHATY_PUPPET="wechaty-puppet-padlocal"
export WECHATY_PUPPET_PADLOCAL_TOKEN="your_token"

export WECHATY_PUPPET_SERVER_PORT="8080"
export WECHATY_TOKEN="your_token"

docker run -ti \
  --name wechaty_puppet_service_token_gateway \
  --rm \
  -e WECHATY_LOG \
  -e WECHATY_PUPPET \
  -e WECHATY_PUPPET_PADLOCAL_TOKEN \
  -e WECHATY_PUPPET_SERVER_PORT \
  -e WECHATY_TOKEN \
  -p "$WECHATY_PUPPET_SERVER_PORT:$WECHATY_PUPPET_SERVER_PORT" \
  wechaty/wechaty:latest
~                        
```

运行成功后如下图:


![](https://ai-studio-static-online.cdn.bcebos.com/73f123bb9b37410eb49f834dc512226e014f3f53a77d42888b201833e8ef6942)



# 十一、扫码登录微信

在控制台找到二维码图片链接， 如Online QR Code Image: https://wechaty.js.org/qrcode/xxxxxxx

![](https://ai-studio-static-online.cdn.bcebos.com/d4df34c54c7840beba551e0edba6cc1b957e6a6f869d406eae177873553d837b)


在浏览器中打开并进行扫码登录

![](https://ai-studio-static-online.cdn.bcebos.com/45ea700e91d84b82a19b0f9b00d8920e822fd96f396b414481a4a1b2bcb557b7)


# 十二、对接PaddleHub模型和WeChaty处理逻辑

配置WeChaty相关信息以及安装相关依赖包
```
import os
os.environ['WECHATY_PUPPET']="wechaty-puppet-service"
os.environ['WECHATY_PUPPET_SERVICE_TOKEN']="your token" # 这里填Wechaty token
os.environ['CUDA_VISIBLE_DEVICES']="0"
os.environ['WECHATY_PUPPET_SERVICE_ENDPOINT']="xx.xx.xx.xx:8080" #这里填Wechaty服务端的IP和端口
os.system("pip install wechaty==0.7dev17")
os.system("hub install ernie_gen_ch_med_save") #安装已保存的模型
```

主要代码逻辑，非常简单，就几行代码
```
model = hub.Module(name='animegan_v2_shinkai_33', use_gpu=True)
async def on_message(msg: Message):
    msg_src = msg.text()
    if msg_src.startswith('Q'): #只处理已Q开头的字符串
        results = module.generate(texts=[msg_src[1:]], use_gpu=True, beam_width=1)
        ans = "A" + results[0][0]
        await msg.say(msg_src + "  " + ans)
```

**PS： 可能由于网络原因在aistuido上无法运行，可以切换到本地或者脚本任务运行**

![](https://ai-studio-static-online.cdn.bcebos.com/9bba5859e9f7485cbb68e5a2d04b864db3b1861ec3d44de5b1ad875017bcaabd)


# 十三、效果测试

Q开头表示提问，过滤其他消息， 回复的时候会把问题(Q)和答案(A)一起发送。也可以自己给自己发送消息，以便测试。

![](https://ai-studio-static-online.cdn.bcebos.com/0990a2ec2acb4a2190301e77a76bb4c53a3aad0a93e64e9e9079d1338f414ad8)




至此你的新场景module就已经测试完毕，大家有没有新场景呢？不妨来试试吧！

# 最后，请各位朋友们给个一键三连（喜欢♡、fork〧、关注+）吧，助我上黄金，谢谢朋友们的支持！

**关于作者**

PaddlePaddle开发爱好者

我在AI Studio上获得黄金等级，点亮5个徽章，来互关呀~ https://aistudio.baidu.com/aistudio/personalcenter/thirdview/89442